In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('/Users/yitsung/Desktop/MasterThesis/EDM_Function/BinaryClassfication/')
# sys.path.append('/raid/ytchang/MasterThesis/EDM_Function/BinaryClassfication/')

from EDMfunction_origin import *
# from EDMfunction_OSPS import *

import warnings
warnings.filterwarnings('ignore')

# Data preprocessing

In [2]:
###########################################################################################
TICKER = 2330
TP = 1
TARGET = f'y_{TP}'

data = pd.read_csv('/Users/yitsung/Desktop/MasterThesis/data/TaiwanStockData_Top100_EMA')
# data = pd.read_csv('/raid/ytchang/MasterThesis/data/TaiwanStockData_Top100_EMA')
###########################################################################################

In [3]:
# import data #
ticker_data = data[data['ticker']==TICKER].reset_index(drop=True)
ticker_data = ticker_data.drop(columns=['ticker'])

# SMA-P/P, binary classification #
ticker_data[f'y_{TP}'] = ticker_data['close'].rolling(window=TP).mean()
ticker_data[f'y_{TP}'] = ticker_data[f'y_{TP}'].shift(-TP)
ticker_data = ticker_data.dropna().reindex()
ticker_data[f'y_{TP}'] = ((ticker_data[f'y_{TP}'] - ticker_data['close']) >= 0).astype(int)

origi_data = ticker_data.copy() # Dataframe for validation

# Restore to a dataframe without disclosing future information. #
ticker_data[f'y_{TP}'] = ticker_data[f'y_{TP}'].shift(TP)
ticker_data = ticker_data.dropna().reset_index(drop=True)

In [4]:
# Splite Library and Prediction #
Library = ticker_data[ticker_data['Date'] <= '2023-06-30'] # the last prediction from Library is 6/30
Prediction = ticker_data[(ticker_data['Date'] >= '2023-07-01')&(ticker_data['Date'] <= '2023-10-31')] 

# EDM pipeline

In [5]:
##############################
KN = 5
EMAX = 10
MAXLAG = 10
THETA_SEQ = [1,2,4,7,11,16,22]
##############################

In [6]:
### Create a dataframe for prediction results. ###
Date = origi_data['Date'][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-10-31')].reset_index(drop=True)
Observations = origi_data[TARGET][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-10-31')].reset_index(drop=True)

EDM_result = pd.DataFrame(Date)
EDM_result['Observations'] = Observations
EDM_result['Predictions'] = None

### Starting from Pred[0] (include searching views for EDM) ###
th=0
Lib_Pred_df = ConcateLibPred(Library=Library, Prediction=Prediction, th=th)
Lib_Pred_df = DataNormalize(Lib_Pred_df=Lib_Pred_df)

formatted_columns, train_feature = EDM_FeatureProcessing(data=Lib_Pred_df, target=TARGET)
TargetOED, TargetOED_rho = EDM_TargetOED(data=Lib_Pred_df, target=TARGET, valid_interval=60)
f_selec = EDM_FeatureSelection(data=Lib_Pred_df, target=TARGET, 
                               train_feature=train_feature, TargetOED=TargetOED, 
                               E_max=EMAX)
Embed_df, ML_df = EDM_CreateNewDF(data=Lib_Pred_df, target=TARGET, f_selec=f_selec, 
                                  max_lag=MAXLAG, th=th)
rs_score = EDM_RandomSimplex(Embed_df=Embed_df, target=TARGET, targetOED=TargetOED, 
                             valid_interval=60, kmax=10000, kn=KN)
dmatrix = EDM_WeightedDistanceMatrix(Embed_df=Embed_df, rs_score=rs_score)
result_ls, theta, param = EDM_ModelSelection(ML_df=ML_df, target=TARGET, dmatrix=dmatrix, 
                                             theta_seq=THETA_SEQ, Tp=TP)
model = EDM_TrainningModel(ML_df=ML_df, target=TARGET, dmatrix=dmatrix, 
                           theta=theta, param=param, Tp=TP)

X_pred = np.array(ML_df.iloc[-1]).reshape(1, -1)
y_pred = model.predict(X_pred)
y_pred = y_pred[0]

EDM_result.loc[th, 'Predictions'] = y_pred
# print(f"{EDM_result['Date'][th]}: finished")

### Predict the remaining ###
for th in range(1, len(Prediction)):
    Library = UnderSampling(Library=Library, target=TARGET) # only for EDMfunction_origin
    Lib_Pred_df = ConcateLibPred(Library=Library, Prediction=Prediction, th=th)
    Lib_Pred_df = DataNormalize(Lib_Pred_df=Lib_Pred_df)
    Embed_df, ML_df = EDM_CreateNewDF(data=Lib_Pred_df, target=TARGET, f_selec=f_selec, 
                                      max_lag=MAXLAG, th=th)
    dmatrix = EDM_WeightedDistanceMatrix(Embed_df=Embed_df, rs_score=rs_score)
    model = EDM_TrainningModel(ML_df=ML_df, target=TARGET, dmatrix=dmatrix, 
                               theta=theta, param=param, Tp=TP)

    X_pred = np.array(ML_df.iloc[-1]).reshape(1, -1)
    y_pred = model.predict(X_pred)
    y_pred = y_pred[0]

    EDM_result.loc[th, 'Predictions'] = y_pred
    # print(f"{EDM_result['Date'][th]}: finished")

EDM_result.dropna(inplace=True)
ACC = len(EDM_result[EDM_result['Predictions'] == EDM_result['Observations']]) / len(EDM_result['Observations'])
print(f'\n ACC: {ACC}\n')

  Theta     Score                                              Param
0     1  0.525296  [{'C': 50, 'fit_intercept': True, 'intercept_s...
1     2  0.528643  [{'C': 100, 'fit_intercept': True, 'intercept_...
2     4  0.520239  [{'C': 100, 'fit_intercept': True, 'intercept_...
3     7  0.521849  [{'C': 10, 'fit_intercept': True, 'intercept_s...
4    11  0.511822  [{'C': 0.1, 'fit_intercept': True, 'intercept_...
5    16  0.511822  [{'C': 1, 'fit_intercept': True, 'intercept_sc...
6    22  0.511822  [{'C': 10, 'fit_intercept': True, 'intercept_s...

 ACC: 0.4939759036144578



In [7]:
# EDM_result.to_csv(f'{TICKER}_MDRSmap_Tp={TP}_ResultOnce.csv', index=False)
EDM_result.head()

,Date,Observations,Predictions
0,2023-07-03,1,0.0
1,2023-07-04,0,1.0
2,2023-07-05,0,0.0
3,2023-07-06,1,0.0
4,2023-07-07,1,1.0


# Test 5 times on one stock

In [8]:
###########################################################################################
TICKER = 2330
TP = 1
TARGET = f'y_{TP}'

data = pd.read_csv('/Users/yitsung/Desktop/MasterThesis/data/TaiwanStockData_Top100_EMA')
# data = pd.read_csv('/raid/ytchang/MasterThesis/data/TaiwanStockData_Top100_EMA')
###########################################################################################

##############################
KN = 5
EMAX = 10
MAXLAG = 10
THETA_SEQ = [1,2,4,7,11,16,22]
##############################

In [9]:
### Create whole test dataframe ###
whole_result = pd.DataFrame()

for num in range(5):

    ### import data ###
    ticker_data = data[data['ticker']==TICKER].reset_index(drop=True)
    ticker_data = ticker_data.drop(columns=['ticker'])

    # SMA-P/P, binary classification #
    ticker_data[f'y_{TP}'] = ticker_data['close'].rolling(window=TP).mean()
    ticker_data[f'y_{TP}'] = ticker_data[f'y_{TP}'].shift(-TP)
    ticker_data = ticker_data.dropna().reindex()
    ticker_data[f'y_{TP}'] = ((ticker_data[f'y_{TP}'] - ticker_data['close']) >= 0).astype(int)

    origi_data = ticker_data.copy() # Dataframe for validation

    # Restore to a dataframe without disclosing future information. #
    ticker_data[f'y_{TP}'] = ticker_data[f'y_{TP}'].shift(TP)
    ticker_data = ticker_data.dropna().reset_index(drop=True)

    # Splite Library and Prediction #
    Library = ticker_data[ticker_data['Date'] <= '2023-06-30'] # the last prediction from Library is 6/30
    Prediction = ticker_data[(ticker_data['Date'] >= '2023-07-01')&(ticker_data['Date'] <= '2023-10-31')] 

    ### Create a dataframe for prediction results. ###
    Date = origi_data['Date'][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-10-31')].reset_index(drop=True)
    Observations = origi_data[TARGET][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-10-31')].reset_index(drop=True)

    EDM_result = pd.DataFrame(Date)
    EDM_result['Observations'] = Observations
    EDM_result['Predictions'] = None

    ### Starting from Pred[0] (include searching views for EDM). ###
    th=0
    Lib_Pred_df = ConcateLibPred(Library=Library, Prediction=Prediction, th=th)
    Lib_Pred_df = DataNormalize(Lib_Pred_df=Lib_Pred_df)

    formatted_columns, train_feature = EDM_FeatureProcessing(data=Lib_Pred_df, target=TARGET)
    TargetOED, TargetOED_rho = EDM_TargetOED(data=Lib_Pred_df, target=TARGET, valid_interval=60)
    f_selec = EDM_FeatureSelection(data=Lib_Pred_df, target=TARGET, 
                                train_feature=train_feature, TargetOED=TargetOED, 
                                E_max=EMAX)
    Embed_df, ML_df = EDM_CreateNewDF(data=Lib_Pred_df, target=TARGET, f_selec=f_selec, 
                                    max_lag=MAXLAG, th=th)
    rs_score = EDM_RandomSimplex(Embed_df=Embed_df, target=TARGET, targetOED=TargetOED, 
                                valid_interval=60, kmax=10000, kn=KN)
    dmatrix = EDM_WeightedDistanceMatrix(Embed_df=Embed_df, rs_score=rs_score)
    result_ls, theta, param = EDM_ModelSelection(ML_df=ML_df, target=TARGET, dmatrix=dmatrix, 
                                                theta_seq=THETA_SEQ, Tp=TP)
    model = EDM_TrainningModel(ML_df=ML_df, target=TARGET, dmatrix=dmatrix, 
                            theta=theta, param=param, Tp=TP)

    X_pred = np.array(ML_df.iloc[-1]).reshape(1, -1)
    y_pred = model.predict(X_pred)
    y_pred = y_pred[0]

    EDM_result.loc[th, 'Predictions'] = y_pred
    # print(f"{EDM_result['Date'][th]}: finished")

    ### Predict the remaining ###
    for th in range(1, len(Prediction)):
        Library = UnderSampling(Library=Library, target=TARGET)
        Lib_Pred_df = ConcateLibPred(Library=Library, Prediction=Prediction, th=th)
        Lib_Pred_df = DataNormalize(Lib_Pred_df=Lib_Pred_df)
        Embed_df, ML_df = EDM_CreateNewDF(data=Lib_Pred_df, target=TARGET, f_selec=f_selec, 
                                        max_lag=MAXLAG, th=th)
        dmatrix = EDM_WeightedDistanceMatrix(Embed_df=Embed_df, rs_score=rs_score)
        model = EDM_TrainningModel(ML_df=ML_df, target=TARGET, dmatrix=dmatrix, 
                                theta=theta, param=param, Tp=TP)
        
        X_pred = np.array(ML_df.iloc[-1]).reshape(1, -1)
        y_pred = model.predict(X_pred)
        y_pred = y_pred[0]

        EDM_result.loc[th, 'Predictions'] = y_pred
        # print(f"{EDM_result['Date'][th]}: finished")

    EDM_result.dropna(inplace=True)
    ACC = len(EDM_result[EDM_result['Predictions'] == EDM_result['Observations']]) / len(EDM_result['Observations'])
    print(f'\n {num+1} of 5 ACC: {ACC}\n')

    whole_result = pd.concat([whole_result, EDM_result], axis=0, ignore_index=True)

# whole_result.to_csv(f'{TICKER}_MDRSmap_Tp={TP}_ResultWhole.csv', index=False)
whole_ACC = len(whole_result[whole_result['Predictions'] == whole_result['Observations']]) / len(whole_result['Observations'])
print(f'\n whole_ACC: {whole_ACC}\n')

  Theta     Score                                              Param
0     1  0.530395  [{'C': 10, 'fit_intercept': True, 'intercept_s...
1     2  0.530395  [{'C': 100, 'fit_intercept': True, 'intercept_...
2     4  0.523658  [{'C': 100, 'fit_intercept': True, 'intercept_...
3     7  0.511822  [{'C': 0.1, 'fit_intercept': True, 'intercept_...
4    11  0.511822  [{'C': 1, 'fit_intercept': True, 'intercept_sc...
5    16  0.511822  [{'C': 1, 'fit_intercept': True, 'intercept_sc...
6    22  0.511822  [{'C': 1, 'fit_intercept': True, 'intercept_sc...

 1 of 5 ACC: 0.4939759036144578

  Theta     Score                                              Param
0     1  0.528728  [{'C': 50, 'fit_intercept': True, 'intercept_s...
1     2  0.528657  [{'C': 100, 'fit_intercept': True, 'intercept_...
2     4  0.530338  [{'C': 100, 'fit_intercept': True, 'intercept_...
3     7  0.516778  [{'C': 100, 'fit_intercept': True, 'intercept_...
4    11  0.511822  [{'C': 1, 'fit_intercept': True, 'intercept_sc...
